In [ ]:
import requests
import json
import os
import pandas as pd

In [ ]:
stock_path = ['Stock_FS.csv']
for a in stock_path:
    if os.path.exists(a):
        os.remove(a)

In [ ]:
def get_web(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.27 Safari/537.36 Edg/99.0.1150.16'}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def get_stock_fs(stock_code, report_date):
    url = 'https://finfo-api.vndirect.com.vn/v4/financial_statements?q=reportType:QUARTER~code:' + stock_code + '~fiscalDate:' + report_date
    data = get_web(url).json()
    page_size = data['totalElements']
    
    #Resize array
    url = url + '&size=' + str(page_size)
    data = get_web(url).json()

    stock_fs_list = []
    for d in data['data']:
        try:
            stock_id = d['code']
            item_code = str(d['itemCode'])
            report_type = d['reportType']
            model_id = str(d['modelType'])
            value = d['numericValue']
            fiscal_date = d['fiscalDate']

            stock_fs_list.append([stock_id, item_code, report_type, model_id, value, fiscal_date])
        except Exception as err:
            print(err)
    
    return stock_fs_list

In [ ]:
def get_industry_fs():
    url = 'https://finfo-api.vndirect.com.vn/v4/financial_models?'
    data = get_web(url).json()
    page_size = data['totalElements']
    
    #Resize array
    url = url + '&size=' + str(page_size)
    data = get_web(url).json()

    industry_fs_list = []
    for d in data['data']:
        try:
            model_id = str(d['modelType'])
            model_code = d['modelTypeName']
            industry_name = d['companyForm']
            item_code = str(d['itemCode'])
            item_name = d['itemVnName']
            display_lvl = d['displayLevel']
            display_ord = d['displayOrder']
            
            industry_fs_list.append([model_id, model_code, industry_name, item_code, item_name, display_lvl, display_ord])
        except Exception as err:
            print(err)
    
    return industry_fs_list

In [ ]:
%%time

stock_code = 'VIB'
report_date = '2021-12-31'

industry_df = pd.DataFrame(get_industry_fs(), columns=['model_id', 'model_code', 'industry_name', 'item_code', 'item_name', 'display_level', 'display_order'])
stock_df = pd.DataFrame(get_stock_fs(stock_code, report_date), columns=['stock_code', 'item_code', 'report_type', 'model_id', 'value', 'fiscal_date'])
bs_df = pd.merge(industry_df, stock_df, how='inner', on=['model_id', 'item_code']).drop(columns=['model_id', 'item_code']).sort_values(by=['fiscal_date', 'stock_code', 'model_code', 'display_order'])
bs_df.to_csv(stock_path[0], encoding='utf-8-sig', index=False)